# SpaceX Launch Success Interactive Dashboard

#### Plotly Dash 

In [ ]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

df = pd.read_csv("space_launch_dash.csv")

max_var = df["Payload Mass (kg)"].max()
min_var = df['Payload Mass (kg)'].min()

app = dash.Dash(__name__)
server = app.server

app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                dcc.Dropdown(id = "site-dropdown",
                                            options = [
                                                    {"label" : "All Sites", "value" : "ALL"},
                                                    {"label" : "CCAFS SLC-40", "value" : "CCAFS SLC-40"},
                                                    {"label" : "KSC LC-39A", "value" : "KSC LC-39A"},
                                                    {"label" : "VAFB SLC-4E", "value" : "VAFB SLC-4E"},
                                                    {"label" : "CCAFS LC-40", "value" : "CCAFS LC-40"}
                                            ],
                                            value = "ALL",
                                            placeholder = "Select a Launch Site",
                                            searchable = True
                                            ),
                                html.Br(),

                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                
                                dcc.RangeSlider(id = 'payload-slider',
                                            min = 0,
                                            max = 10000,
                                            step = 1000,
                                            value = [min_payload, max_payload]
                                ),
                                
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

@app.callback(
    Output(component_id = "success-pie-chart", component_property = "figure"),
    Input(component_id = "site-dropdown", component_property = "value"))
    
def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df["Launch Site"] == entered_site]
    if entered_site == "ALL":
        fig = px.pie(spacex_df, values = "class", names = "Launch Site", title = "Total Successful Launches by Site")
        return fig
    else:
        fig = px.pie(filtered_df, names = "class", title = "Total Successful Launches for site " + entered_site)
        return fig

@app.callback(
    Output(component_id = "success-payload-scatter-chart", component_property = "figure"),
    Input(component_id = "site-dropdown", component_property = "value"),
    Input(component_id = "payload-slider", component_property = "value"))

def get_scatter_plot(entered_site, entered_payload):
    filtered_df = spacex_df[spacex_df["Payload Mass (kg)"].between(entered_payload[0], entered_payload[1])]
    if entered_site == "ALL":
        fig = px.scatter(filtered_df, x = "Payload Mass (kg)", y = "class", color = "Booster Version", title = "Correlation Between Payload Mass and Success for All Launch Sites")
        return fig
    else:
        filtered_df = filtered_df[filtered_df["Launch Site"] == entered_site]
        fig = px.scatter(filtered_df, x = "Payload Mass (kg)", y = "class", color = "Booster Version", title = "Correlation Between Payload Mass and Success for site" + entered_site)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()